# Data Science for Social Justice Workshop: TF-IDF

* * * 


<div class="alert alert-success">  
    
### Learning Objectives 
    
* Understand the TF-IDF algorithm, and how it's used in information retrieval.
* Understand the difference between TF-IDF and simple word counts.
* Use Scikit Learn to extract TF-IDF scores from our text data.
* Use TF-IDF scores to compare posts.
</div>

### Icons Used in This Notebook
🔔 **Question**: A quick question to help you understand what's going on.<br>
💡 **Tip**: How to do something a bit more efficiently or effectively.<br>
⚠️ **Warning:** Heads-up about tricky stuff or common mistakes.<br>
💭 **Reflection**: Reflecting on ethical implications, biases, and social impact in data science.<br>

### Sections
1. [Turning Words into Numbers](#words)
2. [Bag-Of-Words Representation](#bow)
3. [Testing TF-IDF with a toy dataset](#toy)
4. [Using TF-IDF on Reddit datasets](#reddit)
5. [Using TF-IDF to Find Similar Posts](#similar)

<a id='words'></a>

# Turning Words Into Numbers

In the previous notebook, we covered methods to clean and tokenize text into units (like words, bigrams, and trigrams).

For many data science applications, we need a way to convert the content of a text (the list of tokens) into useful numbers that can be used as the features of a model or analysis. One way to do this is to to use a method called Term Frequency-Inverse Document Frequency (TF-IDF). 

This ultimately is a question of **representation**. A text representation is easier for humans to understand, but a numerical representation is easier to perform large scale computational analysis on. The more we change the representation, the more distant our reading of the text becomes. We may gain some benefits from this, but we must also keep in mind what we lose.

## Retrieving the Dataset

We retrieve the data, as we did in the previous notebook:

In [74]:
import os
import pickle
import pandas as pd

In [75]:
with open('../../data/aita_submission_trigrams.pickle', 'rb') as f:
    # Load the object from the file
    aita_trigrams = pickle.load(f)

df = pd.read_csv('../../data/aita_pp.csv')

**Use the `2_TF_IDF_Project.ipynb` notebook to run the TF-IDF operations explained in this notebook on your own data.**

<a id='bow'></a>

# Bag-Of-Words Representation

Before we do TF-IDF, we need to learn something about **bag-of-words (BOW)** models.

The idea of BOW models is to encode the corpus in terms of word frequencies. In a bag-of-words, we take some text, tokenize it, and then tabulate the frequencies of each token. The numerical representation of the text, then, is a vector indicating the frequencies of each token for that text.

For example, if we're considering a Reddit post on r/amitheasshole: 

<img src="../../images/bow.svg"  width="600" height="300">


We take each token from the review, "toss it in a bag", and count up the frequencies of each word. The numerical representation, then, is the vector on the right: the number of appearances of each token. The "bag" here denotes that we are not modeling structure within the text - only the frequencies of the words.


### Document Term Matrix

In most text corpora, we will have many samples or *documents*. For example, in our Reddit dataset, we have many posts. Each post stands on its own as a unique sample: it can be thought of as a unique document in the entire *corpus*. 

Since they are all related to each other, many tokens might be shared across posts. So, when creating the bag-of-words model, we can tokenize across all documents, forming a *vocabulary*. Then, we can represent a single document by which of the tokens in the vocabulary are represented, and their frequency within the document.

If the vocabulary has $V$ tokens, then each document will be encoded in a $V$-dimensional vector. If there are $D$ documents, the entire dataset can be represented in a $D \times V$ matrix, where each row corresponds to the document, and each column corresponds to the token (or "term"). This $D \times V$ matrix is a **document term matrix** (DTM).

Let's consider a simple example. Suppose we have the "documents":

```
["You are at a workshop. Are you ready?",
 "Welcome to Berkeley!",
 "I am teaching a workshop."]
```

The unique (word) tokens in this "corpus", in alphabetical order, are:

```
[a, am, are, at, berkeley, i, ready, teaching, to, welcome, workshop, you]
```

The DTM can be formed by going through each document, ticking off the frequency of each token in each document, and plugging this number into the matrix:

$$
\begin{array}{c|cccccccccccc}
 & \text{a} & \text{am} & \text{are} & \text{at} & \text{berkeley} & \text{i} & \text{ready} & \text{teaching} & \text{to} & \text{welcome} & \text{workshop} & \text{you} \\\hline
\text{Document 1} & 1 & 0 & 2 & 1 & 0 & 0 & 1 & 0 & 0 & 0 & 1 & 1 \\
\text{Document 2} & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 1 & 1 & 0 & 0 \\
\text{Document 3} & 1 & 1 & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 0 & 1 & 0 \\
\end{array}
$$

The numerical representation for each document is a row in the matrix. For example, Document 1 has numerical representation $[1, 0, 2, 1, 0, 0, 1, 0, 0, 0, 1, 1]$.

To create a DTM, we will use the `CountVectorizer` from the package `sklearn`, a heavily used machine learning package.

In [76]:
from sklearn.feature_extraction.text import CountVectorizer

Here is the workflow:

1. We first create a `CountVectorizer` object, and choose specific settings for how we'll go about creating the DTM. Check out the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) to see what options are available.
2. Then, we "fit" this `CountVectorizer` object to the data. In this context, "fitting" consists of establishing a vocabulary of tokens from the documents in your dataset.
3. Finally, we "transform" the data according to the "fitted" `CountVectorizer` object. This means taking our text data and transforming it into a DTM according to the vocabulary established by the "fitting" step.
4. You can do steps 2 and 3 in one fell swoop using a `fit_transform` function.


In [77]:
corpus = [
  'My cat has paws.',
  'Can we let the dog out?',
  'Our dog really likes the cat but the cat does not agree.']
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
test_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
test_df

,agree,but,can,cat,does,dog,has,let,likes,my,not,our,out,paws,really,the,we
0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0
1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,1
2,1,1,0,2,1,1,0,0,1,0,1,1,0,0,1,2,0


Each column in the matrix represents a unique word in the vocabulary, while each row represents the document in our dataset. In this case, we have three sentences (i.e. the document), and therefore we three rows. The values in each cell are the word counts. Note that with this representation, counts of some words could be 0 if the word did not appear in the corresponding document. 


🔔 **Question**: Lets look at the shape of the matrix. How many unique words are there in the matrix?'

In [78]:
X.shape

(3, 17)

Now, we have numbers representing the contents of the documents! Matrices like this are the simplest way to represent texts. However, it biases most frequent words and ends up ignoring rare words which could have helped is in processing our data more efficiently.

Often, we not only want to focus on the frequency of words present in the corpus but also want to know the importance of the words. This is where TF-IDF (term frequency-inverse document frequency) comes in.

# Implementing TF-IDF

TF-IDF, short for **term frequency–inverse document frequency**, is a metric that reflects how important a word is to a **document** in a collection or **corpus**. When talking about text datasets, the dataset is called a corpus, and each datapoint is a document. A document can be a post, a paragraph, a webpage, whatever is considered the individual unit of text for a given datset. A **term** is each unique token in a document (we previously also referred to this as **type**). 

For example in a corpus of sentences, a document might be: `"I went to New York City in New York state."` 

The processed tokens in that document might be: `[went, new_york, city, new_york, state]`.

The document would have four unique terms: `[went, new_york, city, state]`.

The TF-IDF value increases proportionally to the number of times a word appears in the document (the term frequency, or TF), and is offset by the number of documents in the corpus that contain the word (the inverse document frequency, or IDF). This helps to adjust for the fact that some words appear more frequently in general – such as articles and prepositions.

We won't go into much detail about the math behind calculating the TF-IDF (see the D-Lab Text Analysis workshop videos to see more). The key components to remember are:

1. There is one TF-IDF score per unique word and unique document.
2. A high TF-IDF score suggests that word is descriptive of that document.
3. A low TF-IDF score may be because either the word is not frequent in that document, or that it is frequent in many documents in the dataset - either way, it may not be a good descriptor of that document.

The intuition is that if a word occurs many times in one post but rarely in the rest of the corpus, it is probably useful for characterizing that post; conversely, if a word occurs frequently in a post but also occurs frequently in the corpus, it is probably less characteristic of that post.

⚠️ **Warning:**  Word order is still not retained in this type of featurization, since all that is counted is overall frequency of a word in a document. So it is still a **bag-of-words** approach.

<a id='toy'></a>

# Testing TF-IDF with a Toy Dataset

Let's try TF-IDF out with a toy dataset. Here we have three documents about Python, but with different meanings. If we are trying to distinguish between these documents, the word "Python" would not be very useful, since it occurs in all of the documents, but other terms might, like "Monty", "snake", etc.

In [79]:
document1 = """Python is a 2000 made-for-TV horror movie directed by Richard
Clabaugh. The film features several cult favorite actors, including William
Zabka of The Karate Kid fame, Wil Wheaton, Casper Van Dien, Jenny McCarthy,
Keith Coogan, Robert Englund (best known for his role as Freddy Krueger in the
A Nightmare on Elm Street series of films), Dana Barron, David Bowe, and Sean
Whalen."""

document2 = """Python, from the Greek word (πύθων/πύθωνας), is a genus of
nonvenomous pythons[2] found in Africa and Asia. Currently, 7 species are
recognised.[2] A member of this genus, P. reticulatus, is among the longest
snakes known."""

document3 = """Monty Python (also collectively known as the Pythons) are a British 
surreal comedy group who created the sketch comedy television show Monty Python's 
Flying Circus, which first aired on the BBC in 1969. Forty-five episodes were made 
over four series."""

document4 = """Python is an interpreted, high-level, general-purpose programming language. 
Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes 
code readability with its notable use of significant whitespace. Its language constructs and 
object-oriented approach aim to help programmers write clear, logical code for small and 
large-scale projects."""

document5 = """The Colt Python is a .357 Magnum caliber revolver formerly
manufactured by Colt's Manufacturing Company of Hartford, Connecticut.
It is sometimes referred to as a "Combat Magnum". It was first introduced
in 1955, the same year as Smith &amp; Wesson's M29 .44 Magnum. The now discontinued
Colt Python targeted the premium revolver market segment."""

document6 = """The Pythonidae, commonly known simply as pythons, from the Greek word python 
(πυθων), are a family of nonvenomous snakes found in Africa, Asia, and Australia. 
Among its members are some of the largest snakes in the world. Eight genera and 31
species are currently recognized."""

test_list = [document1, document2, document3, document4, document5, document6]

## Using `CountVectorizer()`

Let's use `CountVectorizer()` again, and customize it a bit. 

We will set the variable `cv` to an instance of `CountVectorizer()`, and change two parameters:
    - Set `max_df` (max document frequency) to get rid of words that appear in more than 85% of the corpus. 
    - Set `stop_words` to include the English stopword list, in order to leave those stopwords out of the calculations as well.
    
Check the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) if you need help on how to change these parameters!

In [80]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_df=0.85, stop_words='english')

Now let's run `fit_transform()` on our `test_list` with these settings. We'll also create a dataframe with all the word counts.

In [81]:
word_count_vector = cv.fit_transform(test_list)
pd.DataFrame(word_count_vector.toarray(), columns=cv.get_feature_names_out())

,1955,1969,1991,2000,31,357,44,actors,africa,aim,...,wil,william,word,world,write,year,zabka,πυθων,πύθων,πύθωνας
0,0,0,0,1,0,0,0,1,0,0,...,1,1,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,1
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
4,1,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,1,0,0,0,1,0,...,0,0,1,1,0,0,0,1,0,0


🔔 **Question**: How many documents and unique words are in this dataset?

## Using `TfidfTransformer`

Next, we need to compute the inverse document frequency values. We'll call [`tfidf_transformer.fit()`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html) on the word counts we computed earlier.


In [82]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer() 
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

To get a glimpse of how the IDF values look, let's put these into a DataFrame and sort by weights. Remember, a low IDF indicates something that is less unique.

In [83]:
# Print IDF values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names_out(), columns=["idf_weights"]) 
# Sort ascending 
df_idf.sort_values(by=['idf_weights'])

,idf_weights
known,1.336472
pythons,1.559616
greek,1.847298
nonvenomous,1.847298
created,1.847298
...,...
film,2.252763
films,2.252763
flying,2.252763
fame,2.252763


Notice that the words "python" and "in" have the lowest IDF values. This is expected: these words appear in each and every document in our collection. The lower the IDF value of a word, the less unique it is to any particular document.

Now that we have the idf values, we can compute the TF-IDF scores for our set of documents using `.transform()`

In [84]:
tf_idf_vector = tfidf_transformer.transform(word_count_vector)

By invoking `tfidf_transformer.transform()` we are computing the TF-IDF scores for our docs. Internally, this is weighting TF scores by their IDF scores, so that the more unique a word, the more its frequency counts in a given document.

Let’s print the TF-IDF values of the first document to see if it makes sense. We place the TF-IDF scores from the third document into a `pandas` data frame and sort it in descending order of scores. We can replace the index in `tf_idf_vector[]` to select a different document to check.

In [85]:
feature_names = cv.get_feature_names_out() 
  
# Print the scores 
df_test = pd.DataFrame(tf_idf_vector[5].T.todense(), index=feature_names, columns=["tfidf"]) 
df_test.sort_values(by=["tfidf"], ascending=False)

,tfidf
snakes,0.364650
world,0.222344
commonly,0.222344
simply,0.222344
recognized,0.222344
...,...
englund,0.000000
emphasizes,0.000000
elm,0.000000
discontinued,0.000000


What are the most distinctive words for document 3 (Highest TF-IDF scores?) Does it made sense given the document and corpus?

<a id='reddit'></a>

# Using TF-IDF on Reddit Datasets

Now that we have a good grasp of how TF-IDF is calculated, let's perform this method on our Reddit data. This time, to save time, we will be using `scikit-learn`'s [`TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html?highlight=tfidfvectorizer#sklearn.feature_extraction.text.TfidfVectorizer). It is a class that basically allows us to create a matrix of word counts (what we just did with `CountVectorizer`), and immediately transform them into TF-IDF values.

We simply instantiate an object of the `TfidfVectorizer`. Then, we run it by applying the `fit_transform()` method to our corpus.

Note that we are setting `max_features` to 1000. This means we will only create tf-idf values for the top-1000 most-frequent terms in our corpus. In other words, our vocabulary will be 1000 in length.

💭 **Reflection**: Think about why getting rid of infrequent terms could be helpful, and what the potential risks could be.

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Settings that you use for count vectorizer will go here
tfidf_vectorizer = TfidfVectorizer(max_df=0.85,
                                   max_features=1000,
                                   decode_error='ignore',
                                   stop_words='english',
                                   smooth_idf=True,
                                   use_idf=True)


# Fit and transform the texts
tfidf = tfidf_vectorizer.fit_transform(df['pp_text'])


The `tfidf` object we created is a so-called **sparse matrix**. We will first need to convert this into something we can read and make use of.

In [89]:
# "sparse matrix"
tfidf

<16309x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 1170547 stored elements in Compressed Sparse Row format>

We can put this sparse matrix into a DataFrame. We first convert `tfidf` into a **dense matrix** using the `todense()` method, then pipe that into a `pd.DataFrame` method. 

In [90]:
# Place TF-IDF values in a DataFrame
tfidf_df = pd.DataFrame(tfidf.todense(), columns=tfidf_vectorizer.get_feature_names_out().ravel())

Let's have a look:

In [91]:
tfidf_df.head()

,000,50,able,absolutely,accept,accepted,accident,account,accused,act,...,year,year_old,years,yelled,yelling,yes,yesterday,young,younger,youngest
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.033219,0.091352,0.055321,0.0,0.0,0.092689,0.000000,0.000000,0.0,0.000000
1,0.0,0.0,0.0,0.093159,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.051708,0.0,0.0,0.000000,0.000000,0.098837,0.0,0.121818
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.126057,0.000000,0.0,0.000000
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.172328,0.000000,0.143494,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000


Note that this `tfidf_idf` is a lot like a Document-Term Matrix, except with TF-IDF counts! The columns represent every term in our 1000-word vocabulary. The rows represent documents in which these words appear. 

Note that most values are 0. This is because most words in our vocabulary actually don't show up in most documents.

We can retrieve the highest TF-IDF values across documents in this DataFrame by just summing all TF-IDF values, and then calling `.sort_values()` on our DataFrame.

In [92]:
# Highest TF-IDF values across documents
tfidf_df.sum().sort_values(ascending=False)

said             900.467902
told             838.143205
like             745.433120
wife             634.859104
got              584.786955
                    ...    
tomorrow          41.025046
jack              41.005889
lawyer            40.974490
understanding     40.879688
anna              38.388930
Length: 1000, dtype: float64

<a id='similar'></a>

## Using TF-IDF to Find Similar Posts

We can use TF-IDF to work out the similarity between any pair of documents. So given one post or comment, we could see which posts or comments are most similar. This can be useful if you're trying to find other examples of a pattern you have found and want to explore further.

We'll use the TF-IDF vectorized output from the previous section. Let's choose a particular document, and try and find similar documents.

In [93]:
doc_idx = 25

In [94]:
df['selftext'].iloc[doc_idx]

'It\'s 6PM on a Friday, store has hundreds of people, there are only 3 registers open and lines are ridiculously long. The self checkout has a line that wraps but its the fastest moving line so we wait about 15 minutes in line to check ourselves out.\n\nAfter we check out, a line is forming to exit the building because everyone is waiting for the walmart receipt checker to glance at their receipt.\n\nI\'m already frustrated because of the wait so I skip the line, and the checker anxiously tries to get my attention and loudly says "SIR I NEED TO CHECK YOUR RECEIPT!", I respond with a loud(because its loud in the store) "NO THANK YOU", and walk out of the building.  People start following my example.\n\nThus ensues a 15 minute fight in the car with the wife because she feels I made a scene.\n\nEDIT: Well this turned out well.'

🔔 **Question:** What is this post about?

Let's have a quick look at the TF-IDF scores for the words in this submission to see if these words are indeed typical for this particular submission. 

In [95]:
tfidf_df.loc[doc_idx].sort_values(ascending=False)

line        0.613340
check       0.357541
loud        0.274531
store       0.262149
wait        0.234665
              ...   
friendly    0.000000
friends     0.000000
fuck        0.000000
fucking     0.000000
youngest    0.000000
Name: 25, Length: 1000, dtype: float64

🔔 **Question:** Do the distinctive words have to do with the topic of the post?

## Calculating similarity 

Now let's find the closest posts to this one. The fact that our documents are now in a vector space allows us to make use of mathematical similarity metrics.

**Cosine similarity** is one metric used to measure how similar the documents are irrespective of their size. Mathematically, it measures the cosine of the angle between two vectors projected in a multi-dimensional space. It is equal to 1 if the documents are the same, and decreases to 0 the more dissimilar they are.

We can use a cosine similarity function from `sklearn` to calculate the cosine similarity between each pair of documents:

In [96]:
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity(tfidf)
similarities.shape

(16309, 16309)

We can put the text and scores in a dataframe, and sort by the score:

In [97]:
similar_df = pd.DataFrame({
    'text': df['selftext'].values,
    'score': similarities[doc_idx]}).sort_values('score', ascending=False)

The top document will be the document itself (it's going to have a similarity of 1 with itself). So we look at the next document.

In [98]:
similar_df['text'].iloc[0]

'It\'s 6PM on a Friday, store has hundreds of people, there are only 3 registers open and lines are ridiculously long. The self checkout has a line that wraps but its the fastest moving line so we wait about 15 minutes in line to check ourselves out.\n\nAfter we check out, a line is forming to exit the building because everyone is waiting for the walmart receipt checker to glance at their receipt.\n\nI\'m already frustrated because of the wait so I skip the line, and the checker anxiously tries to get my attention and loudly says "SIR I NEED TO CHECK YOUR RECEIPT!", I respond with a loud(because its loud in the store) "NO THANK YOU", and walk out of the building.  People start following my example.\n\nThus ensues a 15 minute fight in the car with the wife because she feels I made a scene.\n\nEDIT: Well this turned out well.'

In [99]:
similar_df['text'].iloc[1]

'I was in the TSA Precheck line where people are generally cognizant of the regulations. Unfortunately, a woman with her 3 children somehow ended up in this line. I have no patience for people who hold up the TSA screening line. This woman had to fish out all her feeding bottles from her 3 or 4 poorly organized bags and empty them before putting her belongings through security screening. As a result, she held up the security line for a long time. I finally had it and told her she should have been aware of the rules about liquids before getting in line and should have prepared for security screening before getting in line. I suggested that she step aside and let other travelers through and my suggestion was met with a few cheers. \n\nShe seemed really embarrassed but didn’t apologize at all. After I passed the screen I mentioned to the TSA agent he should manage the queue better. When a traveler was holding up the line he needed to step in, and he failed. He apologized and I moved on. I

💭 **Reflection**: Think about why this document seems similar, in terms of TF-IDF scores, to the one we pulled out first. Do you agree with this calculation of similarity? 


## Using TF-IDF to Find Posts

TF-IDF has been used in search algorithms. This makes sense: after all, it can tell you which words are uncommonly frequent in some text. In this sense, TF-IDF can act as a keyword or topic identifier.

For instance, if we would want to look for a text in our DF that has a high TF-IDF score for the word "husband", we could create a **boolean mask** of our `tfidf_df`. Below, we only select those rows where the column "husband" has a TF-IDF score higher than `.5`. We can then use that mask to subset our original `df`! This is because the rows in both DataFrames refer to the same thing: our Reddit posts.

In [100]:
# Subsetting one DF with the mask of another DF
tfidf_husband_df = df[tfidf_df['husband'] > .5]
tfidf_husband_df.head(3)

,Unnamed: 0,idstr,created,author,title,selftext,score,num_comments,nsfw,flair_text,pp_text
127,130,t3_ac4zea,2019-01-03 11:46:31,mmehex,AITA for not wanting my husband’s best friend ...,My husband and I just got into (another) fight...,9595.0,1272.0,0.0,Not the A-hole,husband got fight best_friend comes wednesday ...
3257,3672,t3_dwdbhc,2019-11-14 18:07:36,wifeindilemma87,WIBTA if lie to my husband about having a misc...,I 32(F) and my husband 41(M) have been togethe...,10507.0,729.0,1.0,Asshole,32(f husband 41(m years married months marriag...
3307,3729,t3_dy6ntg,2019-11-18 17:54:53,throwaway2340821,AITA for referring to my soon-to-be-ex husband...,To make this very short:\n\nMy husband and I a...,1584.0,533.0,0.0,NaN,short husband divorcing husband rampant infide...


We can now get the "selftext" column from this new subsetted DataFrame, and then get the first post just to have a look:

In [101]:
print(tfidf_husband_df['selftext'].iloc[0])

My husband and I just got into (another) fight because his best friend who comes over every Wednesday is (again) staying the night. 

My husband and this friend have had a Wednesday night “date” where the guy comes over to our house to “jam” every week for the last two years, and I’m getting to the point that I want to hit the guy over the head with whatever is closest at hand every time he is around. 

Buckle up. This is probably going to be a long one.... if you make it through all of this you are my hero. 

At first he was supposed to come over just for a few hours on Wednesday nights to play guitar with my husband while I was at work. Over time I have stopped working late Wednesday’s but the jam session continues. Which was fine. However, for the last year they have barely touched the guitars and instead the guy just comes over to get high (on pot). Neither my husband or I get high with him (I don’t enjoy pot and my husband isn’t much into it either, but we aren’t morally against i

## Using TF-IDF Correlations to Explore Biases

Calculating correlations of TF-IDF values can be a useful technique to explore relationships between words or terms in a corpus. By analyzing the correlations, we can identify whether certain words tend to appear together more frequently or if they are related to similar topics. 

Pandas allows us to trace pairwise correlations in a DataFrame using the `corr()` method. Note that this creates a new DataFrame that is square and symmetric. Each element in the resulting DataFrame represents the correlation coefficient between all terms.

In [102]:
corr = tfidf_df.corr()

In [103]:
corr

,000,50,able,absolutely,accept,accepted,accident,account,accused,act,...,year,year_old,years,yelled,yelling,yes,yesterday,young,younger,youngest
000,1.000000,0.014377,0.036283,0.000376,0.005287,0.008640,0.028046,0.027706,-0.003072,-0.008839,...,0.043077,-0.012642,0.043293,-0.009155,-0.017503,-0.017856,-0.009682,0.007532,-0.002259,-0.001395
50,0.014377,1.000000,0.016515,0.007029,-0.006558,-0.009143,-0.005214,0.011817,0.001421,-0.007654,...,0.015839,-0.007902,0.023961,-0.007048,-0.008024,-0.009555,-0.016231,-0.014519,-0.015548,-0.008701
able,0.036283,0.016515,1.000000,0.005589,0.013768,0.015388,0.006970,0.017831,0.002701,-0.004302,...,0.059214,0.001261,0.037120,-0.020362,-0.011760,-0.007174,-0.017189,-0.009784,0.005171,0.006512
absolutely,0.000376,0.007029,0.005589,1.000000,0.008503,-0.000656,0.001200,-0.009987,0.011931,0.008677,...,-0.004135,0.003631,0.017716,0.004389,0.000836,0.005901,0.001724,0.003441,-0.002589,-0.004397
accept,0.005287,-0.006558,0.013768,0.008503,1.000000,0.047502,-0.002640,0.004808,0.017821,-0.006461,...,0.018149,-0.000199,0.031134,0.000786,-0.010455,0.004086,-0.001204,0.004903,0.009866,0.013919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yes,-0.017856,-0.009555,-0.007174,0.005901,0.004086,-0.009371,-0.005082,-0.001674,0.005657,0.001783,...,-0.007610,-0.009654,-0.000416,-0.000528,0.005917,1.000000,0.015040,-0.011547,0.001677,0.002373
yesterday,-0.009682,-0.016231,-0.017189,0.001724,-0.001204,-0.002561,-0.001881,0.001003,-0.004654,0.018473,...,-0.030369,-0.007890,-0.045551,0.035033,0.020053,0.015040,1.000000,-0.013860,-0.013815,-0.010303
young,0.007532,-0.014519,-0.009784,0.003441,0.004903,0.011951,-0.007233,0.000459,-0.002775,0.007043,...,0.000166,0.038192,0.023275,-0.010103,-0.007983,-0.011547,-0.013860,1.000000,0.049490,0.027594
younger,-0.002259,-0.015548,0.005171,-0.002589,0.009866,0.031537,0.005604,-0.011977,0.025331,-0.006447,...,0.027075,0.033954,0.054361,0.002039,-0.005437,0.001677,-0.013815,0.049490,1.000000,0.070271


We can use this new DataFrame to compare two concepts in our data. We will look for the columns of "husband" and "wife" – two concepts we expect to appear in this dataset, and that we could imagine being related to gender bias. We then sort the values of the resulting DataFrame based on the "wife" columns in descending order, and print the first 20 values. 

These will be the top-20 words that are most strongly correlated to the term "wife". 

In [104]:
corr[['husband','wife']].sort_values(by='husband',ascending=False)[:30]

,husband,wife
husband,1.000000,-0.079171
mil,0.174490,0.025464
married,0.137605,0.107432
son,0.117042,0.091930
baby,0.107849,0.024711
kids,0.101789,0.070056
sil,0.095831,0.028513
laws,0.093360,0.052908
children,0.091397,0.052083
bil,0.091038,0.017188


## 💭 Reflection: 

- Do these related terms make sense? 
- Do you see some terms that could be indicative of a bias towards women in the data? 
- What happens if you change the `by=` sortation to `men`? What words appear now, and how do they make sense?

<div class="alert alert-success">

## ❗ Key Points

* Term Frequency, or TF, reflects how often a unique token appears in a corpus.
* Inverse Document Frequency (IDF) reflects the number of documents in the corpus that contain a term. 
* The TF-IDF score of a word reflects how important that word is to a document in a collection or corpus.
* Methods from `scikit-learn` can be used to generate TF and TF-IDF scores for a given corpus.
* TF-IDF scores can be used to calculate how similar documents are. We can do this with mathematical similarity metrics, such as cosine similarity.
    
</div>